In [139]:
import numpy as np 
import pandas as pd
from pandas import DataFrame as df
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import seaborn as sns
import math
import ray
ray.init(num_cpus= 10, ignore_reinit_error=True)

2023-03-21 11:49:16,335	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.9.7
Ray version:,2.2.0


# Part A. Simulate Data

In [208]:
# define data structure
I = 1000
J = 2
T = 2

# define true parameters 
# (1) Demand Side
alpha_1 = 0.9
alpha_2 = 1.1
gamma = -0.2
delta = 0.5 
beta = 0.99
# (2) Price
omega_0 = 1
omega_1 = 0.9
sigma_p = 0.5
p_1_initial = 0.5
p_2_initial = 0.6

# MC integration
D = 200

In [209]:
def DGP():
    # generate data frame
    # panel frame I X T
    i_list = []
    t_list = []
    for i in range(1,I+1):
        i_temp = list(np.full(T,i))
        t_temp = [k for k in range(1,T+1)]
        i_list = i_list + i_temp
        t_list = t_list + t_temp

    # draw epsilon
    epsilon_0 = np.random.gumbel(loc=0, scale=1, size=2000)
    epsilon_1 = np.random.gumbel(loc=0, scale=1, size=2000)
    epsilon_2 = np.random.gumbel(loc=0, scale=1, size=2000)

    # price process
    p_1_post = omega_0 + omega_1*p_1_initial + np.random.normal(loc=0, scale=sigma_p, size=1)
    p_2_post = omega_0 + omega_1*p_2_initial + np.random.normal(loc=0, scale=sigma_p, size=1)

    df_master = df({"i": i_list, "t": t_list, "epsilon_0": epsilon_0,"epsilon_1": epsilon_1, "epsilon_2": epsilon_2})
    df_master['price_1'] = np.where(df_master['t'] == 1, p_1_initial, p_1_post)
    df_master['price_2'] = np.where(df_master['t'] == 1, p_2_initial, p_2_post)

    ##########################
    # calculation start here
    ##########################
    sigma_linspace = np.linspace(-2*sigma_p, 2*sigma_p, D)
    p1_linspace = omega_0 + omega_1*p_1_initial + sigma_linspace
    p2_linspace = omega_0 + omega_1*p_2_initial + sigma_linspace

    #(1) Expected Utility of Tomorrow When I Choose j=0 Today
    u1_bar_j0 = ((alpha_1 + gamma*p1_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    u2_bar_j0 = ((alpha_2 + gamma*p2_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    EXP_payoff_j0 = np.log(1+np.exp(u1_bar_j0)+np.exp(u2_bar_j0))

    #(2) Expected Utility of Tomorrow When I Choose j=1 Today
    u1_bar_j1 = ((alpha_1 + gamma*p1_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    u2_bar_j1 = ((alpha_2 + gamma*p2_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    EXP_payoff_j1 = np.log(1+np.exp(u1_bar_j1)+np.exp(u2_bar_j1))

    #(3) Expected Utility of Tomorrow When I Choose j=2 Today
    u1_bar_j2 = ((alpha_1 + gamma*p1_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    u2_bar_j2 = ((alpha_2 + gamma*p2_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    EXP_payoff_j2 = np.log(1+np.exp(u1_bar_j2)+np.exp(u2_bar_j2))


    # calculate discounted utility of j=0,1,2 at t=1
    def time1_discounted_utility(epsilon_0, epsilon_1, epsilon_2):
        per_period_j0 = epsilon_0
        per_period_j1 = alpha_1 + gamma*p_1_initial + epsilon_1
        per_period_j2 = alpha_2 + gamma*p_2_initial + epsilon_2

        payoff_j0 = per_period_j0 + beta*EXP_payoff_j0
        payoff_j1 = per_period_j1 + beta*EXP_payoff_j1
        payoff_j2 = per_period_j2 + beta*EXP_payoff_j2

        return payoff_j0, payoff_j1, payoff_j2
    vec_time1_discounted_utility = np.vectorize(time1_discounted_utility)

    df_master_t1 = df_master.loc[df_master['t']==1].copy()
    t1_payoff_value = vec_time1_discounted_utility(df_master_t1['epsilon_0'].values, df_master_t1['epsilon_1'].values, df_master_t1['epsilon_2'].values)
    df_master_t1['V_0'] = t1_payoff_value[0]
    df_master_t1['V_1'] = t1_payoff_value[1]
    df_master_t1['V_2'] = t1_payoff_value[2]

    def max_finder(v_0, v_1, v_2):
        if max(v_0, v_1, v_2) == v_0:
            return 0
        elif max(v_0, v_1, v_2) == v_1:
            return 1
        elif max(v_0, v_1, v_2) == v_2:
            return 2
    vec_max = np.vectorize(max_finder)

    df_master_t1['decision'] = vec_max(df_master_t1['V_0'].values, df_master_t1['V_1'].values, df_master_t1['V_2'].values)
    decision_info = df_master_t1[['i', 'decision']]


    # t2 calculate should incorporate first period's decision
    df_master_t2 = df_master.loc[df_master['t']==2]
    df_master_t2 = pd.merge(df_master_t2, decision_info, left_on='i', right_on='i')

    def t2_payoff_calculate(epsilon_0, epsilon_1, epsilon_2, decision):
        if decision == 0:
            V_0 = epsilon_0
            V_1 = alpha_1 + gamma*p_1_post + epsilon_1
            V_2 = alpha_2 + gamma*p_2_post + epsilon_2
        elif decision == 1:
            V_0 = epsilon_0
            V_1 = alpha_1 + gamma*p_1_post + delta + epsilon_1
            V_2 = alpha_2 + gamma*p_2_post + epsilon_2
        elif decision == 2:
            V_0 = epsilon_0
            V_1 = alpha_1 + gamma*p_1_post + epsilon_1
            V_2 = alpha_2 + gamma*p_2_post + delta + epsilon_2
        return V_0, V_1, V_2

    vec_t2_payoff_calculate = np.vectorize(t2_payoff_calculate)

    t2_payoff_value = vec_t2_payoff_calculate(df_master_t2['epsilon_0'].values, df_master_t2['epsilon_1'].values, df_master_t2['epsilon_2'].values, df_master_t2['decision'].values)
    df_master_t2['V_0'] = t2_payoff_value[0]
    df_master_t2['V_1'] = t2_payoff_value[1]
    df_master_t2['V_2'] = t2_payoff_value[2]
    df_master_t2 = df_master_t2.drop('decision', axis=1)

    df_master_t2['decision'] = vec_max(df_master_t2['V_0'].values, df_master_t2['V_1'].values, df_master_t2['V_2'].values)

    df_master = df_master_t1.copy()
    df_master = df_master.append(df_master_t2)
    df_master = df_master.sort_values('i')

    return df_master    

# Part B. Estimate Demand Parameters

In [210]:
def MLE_obj(theta, df_observable):
    alpha_1 = 0.9
    alpha_2 = 1.1
    gamma, delta = theta
    # Choice probability at t=2 
    previous_decision = df_observable.loc[df_observable['t']==1]
    previous_decision = previous_decision[['i', 'decision']]
    previous_decision.rename(columns={"decision":"t1_decision"}, inplace=True)

    df_observable_t2 = df_observable.loc[df_observable['t']==2].copy()
    df_observable_t2 = pd.merge(df_observable_t2, previous_decision, left_on='i', right_on='i')

    def t2_choice_prob(price_1, price_2, t1_decision):
        exp_0 = 1
        exp_1 = np.exp(alpha_1 + gamma*price_1 + delta*int(t1_decision==1))
        exp_2 = np.exp(alpha_1 + gamma*price_2 + delta*int(t1_decision==2))
        denominator = exp_0 + exp_1 + exp_2 

        prob_0 = exp_0/denominator
        prob_1 = exp_1/denominator
        prob_2 = exp_2/denominator

        return prob_0, prob_1, prob_2
    vec_t2_choice_prob = np.vectorize(t2_choice_prob)

    choice_prob = vec_t2_choice_prob(df_observable_t2['price_1'].values, df_observable_t2['price_2'].values, df_observable_t2['t1_decision'].values)
    df_observable_t2['prob_j0'] = choice_prob[0]
    df_observable_t2['prob_j1'] = choice_prob[1]
    df_observable_t2['prob_j2'] = choice_prob[2]

    df_observable_t2['decision_j0'] = np.where(df_observable_t2['decision']==0, 1, 0)
    df_observable_t2['decision_j1'] = np.where(df_observable_t2['decision']==1, 1, 0)
    df_observable_t2['decision_j2'] = np.where(df_observable_t2['decision']==2, 1, 0)

    df_observable_t2['prob'] = df_observable_t2['prob_j0']*df_observable_t2['decision_j0'] + df_observable_t2['prob_j1']*df_observable_t2['decision_j1'] + df_observable_t2['prob_j2']*df_observable_t2['decision_j2']
    t2_prob_sum = (np.log(df_observable_t2['prob'].values)).sum()

    
    # Choice probability at t=1
    df_observable_t1 = df_observable.loc[df_observable['t']==1].copy()
    sigma_linspace = np.linspace(-2*sigma_p, 2*sigma_p, 100)
    p1_linspace = omega_0 + omega_1*p_1_initial + sigma_linspace
    p2_linspace = omega_0 + omega_1*p_2_initial + sigma_linspace

    #(1) Expected Utility of Tomorrow When I Choose j=0 Today
    u1_bar_j0 = ((alpha_1 + gamma*p1_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    u2_bar_j0 = ((alpha_2 + gamma*p2_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    EXP_payoff_j0 = np.log(1+np.exp(u1_bar_j0)+np.exp(u2_bar_j0))

    #(2) Expected Utility of Tomorrow When I Choose j=1 Today
    u1_bar_j1 = ((alpha_1 + gamma*p1_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    u2_bar_j1 = ((alpha_2 + gamma*p2_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    EXP_payoff_j1 = np.log(1+np.exp(u1_bar_j1)+np.exp(u2_bar_j1))

    #(3) Expected Utility of Tomorrow When I Choose j=2 Today
    u1_bar_j2 = ((alpha_1 + gamma*p1_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    u2_bar_j2 = ((alpha_2 + gamma*p2_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
    EXP_payoff_j2 = np.log(1+np.exp(u1_bar_j2)+np.exp(u2_bar_j2))


    # calculate discounted utility of j=0,1,2 at t=1
    def t1_choice_prob(price_1, price_2):
        exp_0 = beta*EXP_payoff_j0
        exp_1 = alpha_1 + gamma*price_1 + beta*EXP_payoff_j1
        exp_2 = alpha_2 + gamma*price_2 + beta*EXP_payoff_j2
        denominator = exp_0 + exp_1 + exp_2

        prob_0 = exp_0/denominator
        prob_1 = exp_1/denominator
        prob_2 = exp_2/denominator
        
        return prob_0, prob_1, prob_2
    vec_t1_choice_prob = np.vectorize(t1_choice_prob)

    choice_prob = vec_t1_choice_prob(df_observable_t1['price_1'].values, df_observable_t1['price_2'].values)
    df_observable_t1['prob_j0'] = choice_prob[0]
    df_observable_t1['prob_j1'] = choice_prob[1]
    df_observable_t1['prob_j2'] = choice_prob[2]

    df_observable_t1['decision_j0'] = np.where(df_observable_t1['decision']==0, 1, 0)
    df_observable_t1['decision_j1'] = np.where(df_observable_t1['decision']==1, 1, 0)
    df_observable_t1['decision_j2'] = np.where(df_observable_t1['decision']==2, 1, 0)

    df_observable_t1['prob'] = df_observable_t1['prob_j0']*df_observable_t1['decision_j0'] + df_observable_t1['prob_j1']*df_observable_t1['decision_j1'] + df_observable_t1['prob_j2']*df_observable_t1['decision_j2']
    t1_prob_sum = (np.log(df_observable_t1['prob'].values)).sum()

    return -(t1_prob_sum + t2_prob_sum)

In [211]:
@ray.remote
def MLE_bootstrap(df_observable):
    MLE_result = minimize(MLE_obj, [-0.05, 0.4], args=df_observable, method='Nelder-Mead', options={'maxiter':200})
    return MLE_result.x

MLE_list = [MLE_bootstrap.remote(DGP()) for i in range(500)]
MLE_array = np.array(ray.get(MLE_list))

In [212]:
print("gamma hat: ", MLE_array[:,0].mean())
print("delta hat: ", MLE_array[:,1].mean())

gamma hat:  -0.19014584286991257
delta hat:  0.6203958722419447


In [184]:
########################
# don't run this part
########################
# old result 
print("alpha_1 hat: ", MLE_array[:,0].mean())
print("alpha_2 hat: ", MLE_array[:,1].mean())
print("gamma hat: ", MLE_array[:,2].mean())
print("delta hat: ", MLE_array[:,3].mean())


alpha_1 hat:  2.194625335722964
alpha_2 hat:  3.0734769102841923
gamma hat:  -0.9982934423034984
delta hat:  0.5487704945549948


# Part C. Heterogenous Price Elasticity

In [207]:
gamma_loc = -0.2
gamma_scale = 0.05 

## Part C-(a) Simulate data

In [222]:
def DGP_hetero(gamma_loc, gamma_scale):
    # generate data frame
    # panel frame I X T
    i_list = []
    t_list = []
    for i in range(1,I+1):
        i_temp = list(np.full(T,i))
        t_temp = [k for k in range(1,T+1)]
        i_list = i_list + i_temp
        t_list = t_list + t_temp

    # draw epsilon
    epsilon_0 = np.random.gumbel(loc=0, scale=1, size=2000)
    epsilon_1 = np.random.gumbel(loc=0, scale=1, size=2000)
    epsilon_2 = np.random.gumbel(loc=0, scale=1, size=2000)

    #draw gamma
    sigma_draw = np.random.normal(loc=0, scale=1, size=1000)
    gamma_array = gamma_loc + gamma_scale*sigma_draw
    df_gamma = df({'i': [i for i in range(1, I+1)], 'gamma': gamma_array})

    # price process
    p_1_post = omega_0 + omega_1*p_1_initial + np.random.normal(loc=0, scale=sigma_p, size=1)
    p_2_post = omega_0 + omega_1*p_2_initial + np.random.normal(loc=0, scale=sigma_p, size=1)

    df_master = df({"i": i_list, "t": t_list, "epsilon_0": epsilon_0,"epsilon_1": epsilon_1, "epsilon_2": epsilon_2})
    df_master['price_1'] = np.where(df_master['t'] == 1, p_1_initial, p_1_post)
    df_master['price_2'] = np.where(df_master['t'] == 1, p_2_initial, p_2_post)
    df_master = pd.merge(df_master, df_gamma, left_on='i', right_on='i')

    # simulate decision at t=1 
    df_master_t1 = df_master.loc[df_master['t']==1].copy()

    sigma_linspace = np.linspace(-2*sigma_p, 2*sigma_p, D)
    p1_linspace = omega_0 + omega_1*p_1_initial + sigma_linspace
    p2_linspace = omega_0 + omega_1*p_2_initial + sigma_linspace

    def time1_discounted_utility(epsilon_0, epsilon_1, epsilon_2, gamma):
        #expected utility
        u1_bar_j0 = ((alpha_1 + gamma*p1_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
        u2_bar_j0 = ((alpha_2 + gamma*p2_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
        u1_bar_j1 = ((alpha_1 + gamma*p1_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
        u2_bar_j1 = ((alpha_2 + gamma*p2_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()

        EXP_payoff_j0 = np.log(1+np.exp(u1_bar_j0)+np.exp(u2_bar_j0))
        EXP_payoff_j1 = np.log(1+np.exp(u1_bar_j1)+np.exp(u2_bar_j0))
        EXP_payoff_j2 = np.log(1+np.exp(u1_bar_j0)+np.exp(u2_bar_j1))

        per_period_j0 = epsilon_0
        per_period_j1 = alpha_1 + gamma*p_1_initial + epsilon_1
        per_period_j2 = alpha_2 + gamma*p_2_initial + epsilon_2

        payoff_j0 = per_period_j0 + beta*EXP_payoff_j0
        payoff_j1 = per_period_j1 + beta*EXP_payoff_j1
        payoff_j2 = per_period_j2 + beta*EXP_payoff_j2

        return payoff_j0, payoff_j1, payoff_j2
    vec_time1_discounted_utility = np.vectorize(time1_discounted_utility)

    t1_payoff_value = vec_time1_discounted_utility(df_master_t1['epsilon_0'].values, df_master_t1['epsilon_1'].values, df_master_t1['epsilon_2'].values, df_master_t1['gamma'].values)
    df_master_t1['V_0'] = t1_payoff_value[0]
    df_master_t1['V_1'] = t1_payoff_value[1]
    df_master_t1['V_2'] = t1_payoff_value[2]

    def max_finder(v_0, v_1, v_2):
        if max(v_0, v_1, v_2) == v_0:
            return 0
        elif max(v_0, v_1, v_2) == v_1:
            return 1
        elif max(v_0, v_1, v_2) == v_2:
            return 2
    vec_max = np.vectorize(max_finder)

    df_master_t1['decision'] = vec_max(df_master_t1['V_0'].values, df_master_t1['V_1'].values, df_master_t1['V_2'].values)
    decision_info = df_master_t1[['i', 'decision']]

    # simulate decision at t=2
    df_master_t2 = df_master.loc[df_master['t']==2]
    df_master_t2 = pd.merge(df_master_t2, decision_info, left_on='i', right_on='i')

    def t2_payoff_calculate(epsilon_0, epsilon_1, epsilon_2, gamma, decision):
        if decision == 0:
            V_0 = epsilon_0
            V_1 = alpha_1 + gamma*p_1_post + epsilon_1
            V_2 = alpha_2 + gamma*p_2_post + epsilon_2
        elif decision == 1:
            V_0 = epsilon_0
            V_1 = alpha_1 + gamma*p_1_post + delta + epsilon_1
            V_2 = alpha_2 + gamma*p_2_post + epsilon_2
        elif decision == 2:
            V_0 = epsilon_0
            V_1 = alpha_1 + gamma*p_1_post + epsilon_1
            V_2 = alpha_2 + gamma*p_2_post + delta + epsilon_2
        return V_0, V_1, V_2

    vec_t2_payoff_calculate = np.vectorize(t2_payoff_calculate)

    t2_payoff_value = vec_t2_payoff_calculate(df_master_t2['epsilon_0'].values, df_master_t2['epsilon_1'].values, df_master_t2['epsilon_2'].values, df_master_t2['gamma'].values, df_master_t2['decision'].values)
    df_master_t2['V_0'] = t2_payoff_value[0]
    df_master_t2['V_1'] = t2_payoff_value[1]
    df_master_t2['V_2'] = t2_payoff_value[2]
    df_master_t2 = df_master_t2.drop('decision', axis=1)

    df_master_t2['decision'] = vec_max(df_master_t2['V_0'].values, df_master_t2['V_1'].values, df_master_t2['V_2'].values)

    df_master = df_master_t1.copy()
    df_master = df_master.append(df_master_t2)
    df_master = df_master.sort_values('i')

    return df_master

## Part C-(b) Estimate

In [225]:
df_master = DGP_hetero(-0.2, 0.05)
df_observable = df_master[['i', 't', 'price_1', 'price_2', 'decision']]

In [226]:
df_observable

,i,t,price_1,price_2,decision
0,1,1,0.500000,0.60000,1
0,1,2,1.257205,2.01278,1
2,2,1,0.500000,0.60000,1
1,2,2,1.257205,2.01278,1
4,3,1,0.500000,0.60000,0
...,...,...,...,...,...
1994,998,1,0.500000,0.60000,2
998,999,2,1.257205,2.01278,2
1996,999,1,0.500000,0.60000,2
1998,1000,1,0.500000,0.60000,2


In [242]:
np.random.seed(1)
sigma_draw = np.random.normal(loc=0, scale=1, size=50)

def MLE_obj_hetero(theta, df_observable):
        alpha_1 = 0.9
        alpha_2 = 1.1
        gamma_loc, gamma_scale ,delta = theta        
        gamma_sample = gamma_loc + gamma_scale*sigma_draw

        # Choice probability at t=2 
        previous_decision = df_observable.loc[df_observable['t']==1]
        previous_decision = previous_decision[['i', 'decision']]
        previous_decision.rename(columns={"decision":"t1_decision"}, inplace=True)

        df_observable_t2 = df_observable.loc[df_observable['t']==2].copy()
        df_observable_t2 = pd.merge(df_observable_t2, previous_decision, left_on='i', right_on='i')

        def t2_choice_prob(price_1, price_2, t1_decision):
                exp_0 = 1
                exp_1 = np.exp(alpha_1 + gamma_sample*price_1 + delta*int(t1_decision==1))
                exp_2 = np.exp(alpha_1 + gamma_sample*price_2 + delta*int(t1_decision==2))
                denominator = exp_0 + exp_1 + exp_2 

                prob_0 = exp_0/denominator
                prob_1 = exp_1/denominator
                prob_2 = exp_2/denominator

                return prob_0.mean(), prob_1.mean(), prob_2.mean()
        vec_t2_choice_prob = np.vectorize(t2_choice_prob)

        choice_prob = vec_t2_choice_prob(df_observable_t2['price_1'].values, df_observable_t2['price_2'].values, df_observable_t2['t1_decision'].values)
        df_observable_t2['prob_j0'] = choice_prob[0]
        df_observable_t2['prob_j1'] = choice_prob[1]
        df_observable_t2['prob_j2'] = choice_prob[2]

        df_observable_t2['decision_j0'] = np.where(df_observable_t2['decision']==0, 1, 0)
        df_observable_t2['decision_j1'] = np.where(df_observable_t2['decision']==1, 1, 0)
        df_observable_t2['decision_j2'] = np.where(df_observable_t2['decision']==2, 1, 0)

        df_observable_t2['prob'] = df_observable_t2['prob_j0']*df_observable_t2['decision_j0'] + df_observable_t2['prob_j1']*df_observable_t2['decision_j1'] + df_observable_t2['prob_j2']*df_observable_t2['decision_j2']
        t2_prob_sum = (np.log(df_observable_t2['prob'].values)).sum()

        # Choice probability at t=1
        df_observable_t1 = df_observable.loc[df_observable['t']==1].copy()
        sigma_linspace = np.linspace(-2*sigma_p, 2*sigma_p, 100)
        p1_linspace = omega_0 + omega_1*p_1_initial + sigma_linspace
        p2_linspace = omega_0 + omega_1*p_2_initial + sigma_linspace


        def t1_choice_prob(gamma):
                u1_bar_j0 = ((alpha_1 + gamma*p1_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
                u2_bar_j0 = ((alpha_2 + gamma*p2_linspace)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
                u1_bar_j1 = ((alpha_1 + gamma*p1_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()
                u2_bar_j1 = ((alpha_2 + gamma*p2_linspace + delta)*norm.pdf(sigma_linspace, loc=0, scale=sigma_p)/norm.pdf(sigma_linspace, loc=0, scale=sigma_p).sum()).sum()

                EXP_payoff_j0 = np.log(1+np.exp(u1_bar_j0)+np.exp(u2_bar_j0))
                EXP_payoff_j1 = np.log(1+np.exp(u1_bar_j1)+np.exp(u2_bar_j0))
                EXP_payoff_j2 = np.log(1+np.exp(u1_bar_j0)+np.exp(u2_bar_j1))

                exp_0 = beta*EXP_payoff_j0
                exp_1 = alpha_1 + gamma*p_1_initial + beta*EXP_payoff_j1
                exp_2 = alpha_2 + gamma*p_2_initial + beta*EXP_payoff_j2
                denominator = exp_0 + exp_1 + exp_2

                prob_0 = exp_0/denominator
                prob_1 = exp_1/denominator
                prob_2 = exp_2/denominator

                return prob_0, prob_1, prob_2

        vec_t1_choice_prob = np.vectorize(t1_choice_prob)
        temp_result = vec_t1_choice_prob(gamma_sample)
        df_observable_t1['prob_j0'] = temp_result[0].mean()
        df_observable_t1['prob_j1'] = temp_result[1].mean()
        df_observable_t1['prob_j2'] = temp_result[2].mean()

        df_observable_t1['decision_j0'] = np.where(df_observable_t1['decision']==0, 1, 0)
        df_observable_t1['decision_j1'] = np.where(df_observable_t1['decision']==1, 1, 0)
        df_observable_t1['decision_j2'] = np.where(df_observable_t1['decision']==2, 1, 0)

        df_observable_t1['prob'] = df_observable_t1['prob_j0']*df_observable_t1['decision_j0'] + df_observable_t1['prob_j1']*df_observable_t1['decision_j1'] + df_observable_t1['prob_j2']*df_observable_t1['decision_j2']
        t1_prob_sum = (np.log(df_observable_t1['prob'].values)).sum()

        return -(t1_prob_sum + t2_prob_sum)


In [254]:
@ray.remote
def MLE_bootstrap(df_observable):
    MLE_result = minimize(MLE_obj_hetero, [-0.1, 0.03, 0.4], args=df_observable, method='Nelder-Mead', options={'maxiter':200})
    return MLE_result.x

MLE_list = [MLE_bootstrap.remote(DGP_hetero(-0.2, 0.05)) for i in range(300)]
MLE_array = np.array(ray.get(MLE_list))

(MLE_bootstrap pid=11157) /var/folders/_m/gh9mkkld0rl0y75bn83mfm980000gn/T/ipykernel_5114/2763597789.py:82: RuntimeWarning: invalid value encountered in log


In [255]:
print("gamma loc hat: ", MLE_array[:,0].mean())
print("gamma scale hat: ", MLE_array[:,1].mean())
print("delta hat: ", MLE_array[:,2].mean())

gamma loc hat:  -0.14946991967135861
gamma scale hat:  0.17459453525078134
delta hat:  0.6318406302920649
